# 生成每天的测试结果

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_10=test_day_new(graph_4_10,"graph_4_10")


In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model
ans_4_11=test_day_new(graph_4_11,"graph_4_11")



# 计算IDF字典

这部分的代码使用jinyuan 优化过后的

In [ ]:
# 计算 4-7的IDF值

# share_node_IDF = mp.Manager().dict()

node_set=set()

file_list=[]

file_path="graph_4_4/"
file_l=os.listdir("graph_4_4/")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_4_5/"
file_l=os.listdir("graph_4_5/")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_4_6/"
file_l=os.listdir("graph_4_6/")
for i in file_l:
    file_list.append(file_path+i)


file_path="graph_4_7/"
file_l=os.listdir("graph_4_7/")
for i in file_l:
    file_list.append(file_path+i)

for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']):
                node_set.add(str(jdata['srcmsg']))
            if 'netflow' not in str(jdata['dstmsg']):
                node_set.add(str(jdata['dstmsg'])) 


node_list=list(node_set)


In [ ]:
files_mem_list=[]
for f_path in (file_list):
        f=open(f_path)
        files_mem_list.append(f.read())

In [ ]:
node_IDF={}
for n in tqdm(node_list):  
    find_str=list(eval(n).values())[0]
    IDF=cal_IDF_by_file_in_mem(n,files_mem_list)
    node_IDF[n]=IDF

torch.save(node_IDF,"node_IDF_4_3-7")
print("IDF weight calculate complete!")

# 窗口关系建立函数

In [ ]:
node_IDF_4_4_7=torch.load("node_IDF_4_4-7")
def cal_set_rel(s1,s2,file_list):
    new_s=s1 & s2
    count=0
    for i in new_s:
#     jdata=json.loads(i)
        if 'netflow' not in i and 'glx_alsa_675' not in i and '/data/system/' not in i and '/storage/emulated/' not in i and  '/data/data/com.android' not in i and  '/proc/' not in i and 'nz9885vc.default' not in i :

#         'netflow' not in i
#         and 'usr' not in i and 'var' not in i
            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                IDF=math.log(len(file_list)/(1))
                
            if i in node_IDF_4_4_7.keys():
                IDF4=node_IDF_4_4_7[i]
            else:
                IDF4=math.log(len(file_list_4_4_7)/(1))    
            
#             print(IDF)
            if (IDF+IDF4)>9:
                print("node:",i," IDF:",IDF)
                count+=1
    return count



# 异常检测4-10

In [ ]:
# node_IDF=torch.load("node_IDF_4_10")
node_IDF=torch.load("node_IDF_4_4-7")
y_data_4_10=[]
df_list_4_10=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}
loss_list_4_10=[]


file_l=os.listdir("graph_4_10")
index_count=0
for f_path in (file_l):
    f=open("graph_4_10/"+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_10.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_10/")
    # 为当前窗口提取相关信息 用于和历史窗口进行对比
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'],file_list_4_4_7)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                # 判断两个窗口之间是否存在交集
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
    index_count+=1
    loss_list_4_10.append(loss_avg)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

In [ ]:
name_list=[]
for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
#         for i in name_list:
#             pred_label[i]=1
        print(loss_count)

# 异常检测4-11

In [ ]:
node_IDF=torch.load("node_IDF_4_11")
node_IDF_4_4_7=torch.load("node_IDF_4_4-7")
y_data_4_11=[]
df_list_4_11=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}

loss_list_4_11=[]

file_path_list=[]


file_path="graph_4_11/"
file_l=os.listdir("graph_4_11/")
for i in file_l:
    file_path_list.append(file_path+i)

index_count=0
for f_path in (file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_11.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_11/")
    # 为当前窗口提取相关信息 用于和历史窗口进行对比
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'],file_list_4_4_7)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                # 判断两个窗口之间是否存在交集
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
    index_count+=1
    loss_list_4_11.append(loss_avg)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

In [ ]:
name_list=[]
for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>9:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
#         for i in name_list:
#             pred_label[i]=1
        print(loss_count)

# label生成

In [ ]:
labels={}
    
    
filelist = os.listdir("graph_4_10")
for f in filelist:
    labels["graph_4_10/"+f]=0

filelist = os.listdir("graph_4_11")
for f in filelist:
    labels["graph_4_11/"+f]=0

In [ ]:
attack_list=[
    'graph_4_11/2018-04-11 13:46:38.658000000~2018-04-11 14:02:21.103000000.txt',
    'graph_4_11/2018-04-11 14:02:21.103000000~2018-04-11 14:18:19.001000000.txt',
    'graph_4_11/2018-04-11 14:18:19.001000000~2018-04-11 14:33:38.600000000.txt',
    'graph_4_11/2018-04-11 14:33:38.600000000~2018-04-11 14:49:05.326000000.txt',
    'graph_4_11/2018-04-11 14:49:05.326000000~2018-04-11 15:04:48.749000000.txt',
]
for i in attack_list:
    labels[i]=1

# 统计attack edge数量

In [ ]:
def keyword_hit(line):
    attack_nodes=[
            'shared_files',
        'csb.tracee.27331.27355',
        'netrecon',
#         '/data/data/org.mozilla.fennec_firefox_dev/',
     
#             'firefox',
        '153.178.46.202',
       '111.82.111.27',
        '166.199.230.185',
        '140.57.183.17',
      
        
        ]
    flag=False
    for i in attack_nodes:
        if i in line:
            flag=True
            break
    return flag



files=[
    
        'graph_4_11/2018-04-11 13:46:38.658000000~2018-04-11 14:02:21.103000000.txt',
    'graph_4_11/2018-04-11 14:02:21.103000000~2018-04-11 14:18:19.001000000.txt',
    'graph_4_11/2018-04-11 14:18:19.001000000~2018-04-11 14:33:38.600000000.txt',
    'graph_4_11/2018-04-11 14:33:38.600000000~2018-04-11 14:49:05.326000000.txt',
    'graph_4_11/2018-04-11 14:49:05.326000000~2018-04-11 15:04:48.749000000.txt',
]


In [ ]:
attack_edge_count=0
for fpath in tqdm(files):
    f=open(fpath)
    for line in f:
        if keyword_hit(line):
            attack_edge_count+=1
print(attack_edge_count)

# 可视化分析模块

In [ ]:
replace_dic={
 '/data/data/org.mozilla.fennec_firefox_dev/cache/':'/data/data/org.mozilla.fennec_firefox_dev/cache/*',
     '/data/data/org.mozilla.fennec_firefox_dev/files/':'/data/data/org.mozilla.fennec_firefox_dev/files/*',
    '/system/fonts/':'/system/fonts/*',
    '/data/data/com.android.email/cache/':'/data/data/com.android.email/cache/*',
    '/data/data/com.android.email/files/':'/data/data/com.android.email/files/*',
    'UNNAMED':'UNNAMED:*',
    
}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    return path_name

In [ ]:

original_edges_count=0

graphs=[]
gg=nx.DiGraph()
count=0
# file_list=os.listdir("./test_day_data4_10_emb100/")
for path in tqdm(attack_list):
#     print(path)
    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            count+=1
            l=line.strip()
            jdata=eval(l)
#             temp_key=jdata['srcmsg']+jdata['dstmsg']+jdata['edge_type']
#             if temp_key in train_edge_set:
#                 jdata['loss']=(jdata['loss']-train_edge_set[temp_key]) if jdata['loss']>=train_edge_set[temp_key] else 0  
#             jdata['loss']=abs(jdata['loss']-train_edge_set[temp_key])  if temp_key in train_edge_set else jdata['loss']
            edge_list.append(jdata)
            
        edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True) 
        original_edges_count+=len(edge_list)
        
        loss_list=[]
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean=mean(loss_list)
        loss_std=std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr=loss_mean+1*loss_std
#         thr=-99
        print("thr:",thr)
        for e in edge_list:
            if e['loss']>thr:    
#             if True:  
#                 if "'/home/admin/profile'" in e['srcmsg'] or " '/home/admin/profile'" in e['dstmsg']:
#                     print(e['srcmsg'])
#                     print(e['dstmsg'])
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))),loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
                #不去除重复节点
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
        print(path)
        print("tempg edges:",len(tempg.edges))
        print("tempg nodes:",len(tempg.nodes))
        print("tempg weakly components:",nx.number_weakly_connected_components(tempg))
        
        print("gg edges:",len(gg.edges))
        print("gg nodes:",len(gg.nodes))
        print("gg weakly components:",nx.number_weakly_connected_components(gg))
        print(f"{original_edges_count=}")
#                                 不去除重复节点
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

In [ ]:
def attack_edge_flag(msg):
    attack_edge_type=[
        '/data/data/org.mozilla.fennec_firefox_dev/',
        '/data/data/org.mozilla.fennec_firefox_dev/shared_files',
        '/data/local/tmp',
        'csb.tracee.27331.27355',
        '/data/data/org.mozilla.fennec_firefox_dev/csb.tracee.27331.27355',
        '111.82.111.27',
        '166.199.230.185',
        'glx_alsa_675',
  
    ]
    flag=False
    for i in attack_edge_type:
        if i in msg:
            flag=True
    return flag

# 异常检测结果统计分析

沿用其他数据集